# SGD
数式では
$$W←W-\eta\frac{\partial L}{\partial W}$$のように記される。
SGDは勾配方向へ一定の距離だけ進める単純な方法。  
以下が実装である。

In [2]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
        def update(self, param, grads):
            for key in param.key():
                param[key] -= self.lr * grads[key] #lrはlearning rate(学習係数)を示す。

params,gradsはディクショナリ変数。
## SGDの欠点
関数の形状が伸びた形のものであると、非効率の経路で探索する点。
# momentum 
$$v←\alpha v - \eta\frac{\partial L}{\partial W}$$
$$W←W-v$$
この式は物理に関係したものである。上の式は物体が勾配方向に力を受けることを示す。αは物体が何も力を受けないときに徐々に減速する役割を担う。  
以下が実装

In [ ]:
class Momentum:

    """Momentum SGD"""

    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():                                
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key] 
            params[key] += self.v[key]


これで問題を解くと、ｘ軸方向に受ける力は小さい一方で常に同方向への力を受けるため同方向に一定して加速できる。  
y軸は正と負の力を交互に受けるためお互いに打ち消しあってｙ軸方向の力が安定しない。
# AdaGram
ニューラルネットワークにおいては学習係数が大きすぎても小さすぎてもよくない。  
これを効率的に解決するためには学習係数の減衰というものがある。これは学習が進むにつれて学習係数を小さくするといったものだ。  
$$h←h +\frac{\partial L}{\partial W}・\frac{\partial L}{\partial W}$$
$$W←W - \eta\frac{\partial 1}{\partial \sqrt{h}}\frac{\partial L}{\partial W}$$
hの二乗根分の1を乗算することで学習のスケールを調整する。  
つまり、よく動いたパラメーターの学習係数が次第に小さくなるという学習係数の減衰をパラメーターの要素ごとに行うのだ。  
以下が実装

In [ ]:
class AdaGrad:

    """AdaGrad"""

    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)


# Adam
momentumではボールがお椀を転がるように物理法則に準じた動きをした。  
AdamGradではパラメーターの要素ごとに適応的に更新ステップを調整した。  
これら二つを融合させたのがAdamである。  
この二つが融合することで効率的にパラメーター空間を探索することが期待できる。  
また、ハイパーパラメータの倍悪補正も行われている。
# どの更新手段を使うか
残念なことにすべてにおいて優れていｒ手法はないため、それぞれの特徴に合わせて使うことが求められる。

# 重みの初期値
重みを小さくすることで過学習は起きにくくなる。  
一方で負の数出ない数で最も小さい数の0を重みに設定すると正しい学習が行えない。  
正確には均一の値に重みを設定してはならないのだ。  
なぜなら誤差逆伝播法で均一に重みが更新されてしまうからだ。  
すると重みが対照的な値(重複した値)を持ってしまい、たくさんの重みをもつ必要がなくなってしまうからだ。<br><br>
隠れ層のアクティベーションは難しくて理解ができなかった。  
結論としては、活性化関数にReLUを使うときはHeの初期値を、sigmoidやtanhなどのS字カーブを使うときはXavierの初期値を使うというのが現段階のベストプラクティスである。
# Batch Normalization
強制的にアクティベーションの分布を調整するのがBatchNormalizationである。  
これの利点は
・学習を早く進行できる(学習係数を大きくできる)  
・初期値にそれほど依存しない、初期値に神経質にならなくてよい  
・過学習を抑制する。
# 正則化
機械学習においては過学習が問題となることがよくある  
その理由は大きく二つあげられる。  
・パラメータを大量に持ち表現性の高いモデルであること。  
・訓練データが少ないこと。
# Weight decay
Weight decay は過学習抑制によく使われる。  
これは学習の過程において大きな重みをもつことにペナルティ課すことで過学習を抑制しようというものである。
# Dropout 
ニューラルネットワークが複雑化するとWeight decayのみでは対応が困難になる。  
そこでDropoutという手法が用いられる。訓練時に隠れ層のニューロンをランダムに消去して学習する手段だ。  
消去されたニューロンは信号の伝達が行われなくなる。
# ハイパーパラメータの検証
ニューラルネットワークでは重みやバイアスとは別にハイパーパラメータが多く登場する。  
ここでいうハイパーパラメータとは各層のニューロン数やバッチサイズ、パラメータの更新の際の学習係数やWeight decay などである。
# 検証データ
訓練データ、テストデータとは別に検証データも必要である。なぜならば汎化性能を調べるためにテストデータを用いても、テストデータに対して過度に適応してしまうことがあるからだ。検証データはハイパーパラメータの調整のために用いる。  
テストデータは汎化性能の確認のため、一度だけ用いるのが理想的である。　
# ハイパーパラメータの最適化
ディープラーニングの学習には多くの時間を要するため、筋の悪そうなハイパーパラメータは早い段階で捨てることが重要である。  
ハイパーパラメータの範囲はおおまかにざっくりと設定するのが有効である。ざっくりと指定するとは10<sup>-3</sup>から10<sup>3</sup>のようの10のn乗のスケールで指定することだ。  
やることをまとめると  
ステップ１　ハイパーパラメータの範囲を指定する。  
ステップ２　指定されたハイパーパラメータの範囲からランダムにサンプリングする。
ステップ３　ステップ１でサンプリングされたハイパーパラメータの値を使用して学習を行い、検証データで認識制度を評価する。　
ステップ４　ステップ２，３を100回程度繰り返してそれら認識精度の結果からハイパーパラメータの範囲を狭める。